# 🏥 Drug Classification ML Pipeline
## Building a Reproducible Machine Learning Pipeline for Deployment

This notebook demonstrates how to build a comprehensive ML pipeline for drug classification with continuous integration and deployment (CI/CD) to Hugging Face Spaces.

### Pipeline Overview:
- **Data Folder**: Stores raw and processed data for reproducible training
- **Model Folder**: Contains saved trained models for easy deployment
- **App Folder**: Houses the Gradio web application for model interaction
- **Results Folder**: Stores evaluation metrics and visualizations
- **CI/CD**: Automated training and deployment using GitHub Actions

### Workflow:
```
DATA → PREPROCESSING → TRAINING → EVALUATION → DEPLOYMENT → MONITORING
```

---

## 1. Environment Setup and Repository Configuration

Let's start by setting up our development environment and configuring repository connections.

In [ ]:
# Import required libraries for the entire pipeline
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Machine learning libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.metrics import ConfusionMatrixDisplay

# Model persistence
import joblib

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Environment setup complete!")
print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")

## 2. Project Structure Creation

Create the organized folder structure and initialize essential files for our ML pipeline.

In [ ]:
# Create necessary directories for the ML pipeline
directories = ["data", "Model", "app", "Results", ".github/workflows"]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"📁 Created directory: {directory}")

# Check current project structure
print("\n📂 Current Project Structure:")
for root, dirs, files in os.walk("."):
    level = root.replace(".", "").count(os.sep)
    indent = " " * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * 2 * (level + 1)
    for file in files:
        if not file.startswith('.'):
            print(f"{subindent}{file}")

print("\n✅ Project structure initialized!")

## 3. Data Loading and Preprocessing

Load the drug classification dataset and perform initial exploration.

In [ ]:
# Load the drug classification dataset
try:
    drug_df = pd.read_csv("data/drug200.csv")
    print("✅ Dataset loaded successfully!")
except FileNotFoundError:
    print("❌ Dataset not found. Please ensure drug200.csv is in the data/ folder.")
    # For demonstration, create a sample dataset
    np.random.seed(125)
    n_samples = 200
    
    drug_df = pd.DataFrame({
        'Age': np.random.randint(15, 75, n_samples),
        'Sex': np.random.choice(['M', 'F'], n_samples),
        'BP': np.random.choice(['HIGH', 'LOW', 'NORMAL'], n_samples),
        'Cholesterol': np.random.choice(['HIGH', 'NORMAL'], n_samples),
        'Na_to_K': np.random.uniform(6.2, 38.2, n_samples),
        'Drug': np.random.choice(['drugA', 'drugB', 'drugC', 'drugX', 'DrugY'], n_samples)
    })
    print("📊 Created sample dataset for demonstration")

# Shuffle the dataset for better training
drug_df = drug_df.sample(frac=1, random_state=125).reset_index(drop=True)

print(f"\n📊 Dataset Info:")
print(f"Shape: {drug_df.shape}")
print(f"Columns: {list(drug_df.columns)}")

# Display first few rows
print("\n🔍 First 5 rows:")
drug_df.head()

In [ ]:
# Data exploration and analysis
print("📈 Dataset Statistics:")
print(drug_df.describe(include='all'))

print("\n🎯 Target Distribution:")
print(drug_df['Drug'].value_counts())

print("\n🔍 Data Types:")
print(drug_df.dtypes)

print("\n❓ Missing Values:")
print(drug_df.isnull().sum())

# Visualize data distribution
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Drug Classification Dataset - Exploratory Data Analysis', fontsize=16)

# Age distribution
axes[0, 0].hist(drug_df['Age'], bins=20, alpha=0.7, color='skyblue')
axes[0, 0].set_title('Age Distribution')
axes[0, 0].set_xlabel('Age')
axes[0, 0].set_ylabel('Frequency')

# Sex distribution
drug_df['Sex'].value_counts().plot(kind='bar', ax=axes[0, 1], color=['pink', 'lightblue'])
axes[0, 1].set_title('Gender Distribution')
axes[0, 1].set_ylabel('Count')

# Blood Pressure distribution
drug_df['BP'].value_counts().plot(kind='bar', ax=axes[0, 2], color=['red', 'orange', 'green'])
axes[0, 2].set_title('Blood Pressure Distribution')
axes[0, 2].set_ylabel('Count')

# Cholesterol distribution
drug_df['Cholesterol'].value_counts().plot(kind='bar', ax=axes[1, 0], color=['purple', 'yellow'])
axes[1, 0].set_title('Cholesterol Distribution')
axes[1, 0].set_ylabel('Count')

# Na_to_K distribution
axes[1, 1].hist(drug_df['Na_to_K'], bins=20, alpha=0.7, color='lightgreen')
axes[1, 1].set_title('Na_to_K Ratio Distribution')
axes[1, 1].set_xlabel('Na_to_K Ratio')
axes[1, 1].set_ylabel('Frequency')

# Drug distribution
drug_df['Drug'].value_counts().plot(kind='bar', ax=axes[1, 2], color=['red', 'blue', 'green', 'orange', 'purple'])
axes[1, 2].set_title('Drug Type Distribution')
axes[1, 2].set_ylabel('Count')

plt.tight_layout()
plt.show()

print("✅ Data exploration complete!")

## 4. Model Training Pipeline

Build a comprehensive scikit-learn pipeline with preprocessing and model training.

In [ ]:
# Prepare features and target variables
print("🔧 Preparing data for training...")

# Separate features and target
X = drug_df.drop("Drug", axis=1).values
y = drug_df["Drug"].values

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=125, stratify=y
)

print(f"\n📊 Data Split:")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Show the original column names for reference
print(f"\n📋 Feature columns (by index):")
for i, col in enumerate(drug_df.columns[:-1]):
    print(f"  {i}: {col}")

print("✅ Data preparation complete!")

In [ ]:
# Define preprocessing pipeline
print("🏗️ Building preprocessing pipeline...")

# Define column indices for different transformations
# Based on drug_df columns: Age, Sex, BP, Cholesterol, Na_to_K
cat_col = [1, 2, 3]  # Sex, BP, Cholesterol (categorical)
num_col = [0, 4]     # Age, Na_to_K (numerical)

print(f"Categorical columns (indices): {cat_col}")
print(f"Numerical columns (indices): {num_col}")

# Create preprocessing transformer
transform = ColumnTransformer([
    ("encoder", OrdinalEncoder(), cat_col),
    ("num_imputer", SimpleImputer(strategy="median"), num_col),
    ("num_scaler", StandardScaler(), num_col),
])

# Create complete ML pipeline
pipe = Pipeline(steps=[
    ("preprocessing", transform),
    ("model", RandomForestClassifier(
        n_estimators=100, 
        random_state=125,
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=2
    )),
])

print("✅ Pipeline created successfully!")
print("\n🔍 Pipeline steps:")
for i, (name, step) in enumerate(pipe.steps):
    print(f"  {i+1}. {name}: {type(step).__name__}")

In [ ]:
# Train the model
print("🚂 Training the model...")
print("This may take a few moments...")

# Fit the pipeline
pipe.fit(X_train, y_train)

print("✅ Model training complete!")

# Get model information
model = pipe.named_steps['model']
print(f"\n📊 Model Details:")
print(f"Algorithm: {type(model).__name__}")
print(f"Number of estimators: {model.n_estimators}")
print(f"Max depth: {model.max_depth}")
print(f"Random state: {model.random_state}")

# Feature importance (after preprocessing)
try:
    importances = model.feature_importances_
    print(f"\n🎯 Feature Importances:")
    feature_names = ['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']
    for i, importance in enumerate(importances):
        print(f"  {feature_names[i]}: {importance:.4f}")
except:
    print("Feature importance information not available")

## 5. Model Evaluation and Metrics

Evaluate the model performance and generate comprehensive metrics.

In [ ]:
# Make predictions
print("🔮 Making predictions...")
predictions = pipe.predict(X_test)

# Calculate performance metrics
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average="macro")

print(f"\n📊 Model Performance:")
print(f"Accuracy: {accuracy:.3f} ({accuracy*100:.1f}%)")
print(f"F1 Score (macro): {f1:.3f}")

# Detailed classification report
print(f"\n📋 Detailed Classification Report:")
print(classification_report(y_test, predictions))

# Save metrics to file
metrics_text = f"Accuracy = {round(accuracy, 3)}, F1 Score = {round(f1, 3)}"
with open("Results/metrics.txt", "w") as outfile:
    outfile.write(metrics_text)

print(f"\n💾 Metrics saved to Results/metrics.txt")
print(f"Content: {metrics_text}")

In [ ]:
# Create and display confusion matrix
print("📊 Generating confusion matrix...")

cm = confusion_matrix(y_test, predictions, labels=pipe.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=pipe.classes_)

# Create a comprehensive visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Confusion Matrix
disp.plot(ax=ax1, cmap='Blues', values_format='d')
ax1.set_title("Confusion Matrix", fontsize=14, fontweight='bold')

# Model Performance Metrics Bar Chart
metrics_names = ['Accuracy', 'F1 Score']
metrics_values = [accuracy, f1]
bars = ax2.bar(metrics_names, metrics_values, color=['skyblue', 'lightcoral'], alpha=0.8)
ax2.set_title("Model Performance Metrics", fontsize=14, fontweight='bold')
ax2.set_ylabel("Score")
ax2.set_ylim(0, 1)

# Add value labels on bars
for bar, value in zip(bars, metrics_values):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{value:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig("Results/model_results.png", dpi=120, bbox_inches='tight')
plt.show()

print("✅ Confusion matrix saved to Results/model_results.png")

## 6. Model Persistence

Save the trained pipeline for deployment and future use.

In [ ]:
# Save the trained pipeline using joblib
print("💾 Saving trained model...")

model_path = "Model/drug_pipeline.joblib"
joblib.dump(pipe, model_path)
print(f"✅ Model saved to: {model_path}")

# Verify model can be loaded
print("\n🔍 Verifying model loading...")
try:
    loaded_pipe = joblib.load(model_path)
    print("✅ Model loaded successfully!")
    
    # Test loaded model with a sample prediction
    sample_input = [[30, 'M', 'HIGH', 'NORMAL', 15.4]]
    sample_prediction = loaded_pipe.predict(sample_input)
    print(f"✅ Sample prediction works: {sample_prediction[0]}")
    
except Exception as e:
    print(f"❌ Error loading model: {str(e)}")

# Model file information
import os
model_size = os.path.getsize(model_path) / (1024 * 1024)  # Size in MB
print(f"\n📁 Model file size: {model_size:.2f} MB")

print(f"\n📋 Model Summary:")
print(f"  - Algorithm: Random Forest Classifier")
print(f"  - Features: 5 (Age, Sex, BP, Cholesterol, Na_to_K)")
print(f"  - Classes: {len(pipe.classes_)} ({', '.join(pipe.classes_)})")
print(f"  - Accuracy: {accuracy:.3f}")
print(f"  - F1 Score: {f1:.3f}")
print(f"  - File: {model_path}")

print("\n🎉 Model persistence complete!")

## 7. Gradio Application Development

Create an interactive web application for model deployment (demonstration code).

In [ ]:
# Demonstration of Gradio app structure (actual app is in app/drug_app.py)
print("🌐 Gradio Application Structure:")

app_code = '''
import gradio as gr
import joblib

# Load model
pipe = joblib.load("../Model/drug_pipeline.joblib")

def predict_drug(age, sex, blood_pressure, cholesterol, na_to_k_ratio):
    """Predict drug based on patient features"""
    features = [age, sex, blood_pressure, cholesterol, na_to_k_ratio]
    predicted_drug = pipe.predict([features])[0]
    probabilities = pipe.predict_proba([features])[0]
    
    # Return probability distribution
    prob_dict = {pipe.classes_[i]: float(probabilities[i]) 
                 for i in range(len(pipe.classes_))}
    return prob_dict

# Define interface components
inputs = [
    gr.Slider(15, 74, step=1, label="Age", value=30),
    gr.Radio(["M", "F"], label="Gender", value="M"),
    gr.Radio(["HIGH", "LOW", "NORMAL"], label="Blood Pressure", value="NORMAL"),
    gr.Radio(["HIGH", "NORMAL"], label="Cholesterol", value="NORMAL"),
    gr.Slider(6.2, 38.2, step=0.1, label="Na_to_K Ratio", value=15.0),
]

outputs = [gr.Label(num_top_classes=5, label="Drug Prediction")]

# Create and launch interface
demo = gr.Interface(
    fn=predict_drug,
    inputs=inputs,
    outputs=outputs,
    title="🏥 Drug Classification System",
    description="Predict the most suitable drug based on patient characteristics",
    theme=gr.themes.Soft(),
)

if __name__ == "__main__":
    demo.launch(share=True)
'''

print("📝 Key Components:")
print("  - Interactive sliders for numerical inputs")
print("  - Radio buttons for categorical choices")
print("  - Prediction function with probability output")
print("  - Professional UI with examples")

print("\n🎯 Features:")
print("  - Real-time predictions")
print("  - Confidence scores for all drug types")
print("  - User-friendly interface")
print("  - Example inputs for testing")

print(f"\n📄 Full application code is available in: app/drug_app.py")
print("✅ Gradio app structure demonstration complete!")

## 8. GitHub Actions CI/CD Configuration

Overview of the automated workflow for continuous integration and deployment.

In [ ]:
# CI/CD Pipeline Overview
print("🔄 Continuous Integration/Continuous Deployment Pipeline")
print("=" * 60)

pipeline_steps = {
    "1. Code Push": "Developer pushes code to GitHub main branch",
    "2. Trigger": "GitHub Actions workflow automatically triggered",
    "3. Environment": "Ubuntu latest server with Python 3.9",
    "4. Dependencies": "Install requirements using Makefile",
    "5. Training": "Execute train.py to train model",
    "6. Evaluation": "Generate metrics and confusion matrix",
    "7. Artifacts": "Upload model and results as artifacts",
    "8. Branch Update": "Create update branch with model files",
    "9. HF Deployment": "Deploy to Hugging Face Spaces",
    "10. Release": "Create GitHub release with version tag"
}

for step, description in pipeline_steps.items():
    print(f"{step}: {description}")

print("\n📁 Key Files in CI/CD:")
print("  - .github/workflows/ci.yml: GitHub Actions workflow")
print("  - Makefile: Automation commands")
print("  - requirements.txt: Python dependencies")
print("  - train.py: Model training script")

print("\n🔐 Required Secrets:")
print("  - GITHUB_TOKEN: For repository operations")
print("  - HF_TOKEN: For Hugging Face deployment")

print("\n🎯 Benefits:")
print("  - Automated model training on data changes")
print("  - Consistent deployment process")
print("  - Version control for models")
print("  - Automatic testing and validation")

print("\n✅ CI/CD configuration overview complete!")

## 9. Hugging Face Deployment Setup

Configuration for deploying to Hugging Face Spaces.

In [ ]:
# Hugging Face Deployment Configuration
print("🤗 Hugging Face Spaces Deployment")
print("=" * 40)

deployment_info = {
    "Space URL": "https://huggingface.co/spaces/Mritula123/Mlmodeldrug",
    "SDK": "Gradio 4.16.0",
    "App File": "drug_app.py",
    "License": "Apache 2.0",
    "Model Format": "joblib",
}

print("📋 Deployment Configuration:")
for key, value in deployment_info.items():
    print(f"  {key}: {value}")

print("\n📁 Files Deployed to HF Spaces:")
print("  - app/drug_app.py: Main application")
print("  - app/README.md: HF metadata and description")
print("  - app/requirements.txt: Dependencies")
print("  - Model/drug_pipeline.joblib: Trained model")
print("  - Results/: Performance metrics and plots")

print("\n🔧 Deployment Commands (from Makefile):")
deployment_commands = [
    "huggingface-cli login --token $HF_TOKEN",
    "huggingface-cli upload Mritula123/Mlmodeldrug ./app --repo-type=space",
    "huggingface-cli upload Mritula123/Mlmodeldrug ./Model --repo-type=space",
    "huggingface-cli upload Mritula123/Mlmodeldrug ./Results --repo-type=space"
]

for i, cmd in enumerate(deployment_commands, 1):
    print(f"  {i}. {cmd}")

print("\n🎯 Deployment Benefits:")
print("  - Public access to the model")
print("  - Automatic environment setup")
print("  - Version control for deployments")
print("  - Easy sharing and collaboration")

print("\n✅ Hugging Face deployment configuration complete!")

## 10. Pipeline Testing and Validation

Test the complete pipeline and validate the deployment.

In [ ]:
# Complete Pipeline Validation
print("🧪 Pipeline Testing and Validation")
print("=" * 40)

# Test model with various input scenarios
test_cases = [
    {
        "name": "Young Male - High BP",
        "input": [25, "M", "HIGH", "NORMAL", 15.4],
        "expected_features": "Young patient with hypertension"
    },
    {
        "name": "Middle-aged Female - Normal",
        "input": [45, "F", "NORMAL", "NORMAL", 10.2],
        "expected_features": "Middle-aged patient with normal vitals"
    },
    {
        "name": "Senior Male - High Cholesterol",
        "input": [65, "M", "LOW", "HIGH", 25.8],
        "expected_features": "Senior patient with cholesterol issues"
    }
]

print("🎯 Testing Model Predictions:")
for i, test_case in enumerate(test_cases, 1):
    prediction = pipe.predict([test_case["input"]])[0]
    probabilities = pipe.predict_proba([test_case["input"]])[0]
    confidence = max(probabilities)
    
    print(f"\n  Test {i}: {test_case['name']}")
    print(f"    Input: {test_case['input']}")
    print(f"    Prediction: {prediction}")
    print(f"    Confidence: {confidence:.3f}")
    print(f"    Context: {test_case['expected_features']}")

# Validate file structure
print("\n📁 Validating File Structure:")
required_files = [
    "train.py",
    "Makefile", 
    "requirements.txt",
    "app/drug_app.py",
    "app/README.md",
    "app/requirements.txt",
    ".github/workflows/ci.yml",
    "Model/drug_pipeline.joblib",
    "Results/metrics.txt",
    "Results/model_results.png"
]

for file_path in required_files:
    if os.path.exists(file_path):
        print(f"  ✅ {file_path}")
    else:
        print(f"  ❌ {file_path} (missing)")

# Summary of the complete pipeline
print(f"\n🎉 Pipeline Summary:")
print(f"  - Model trained successfully: ✅")
print(f"  - Model accuracy: {accuracy:.3f}")
print(f"  - Model saved: ✅")
print(f"  - Results generated: ✅")
print(f"  - Gradio app created: ✅")
print(f"  - CI/CD configured: ✅")
print(f"  - HF deployment ready: ✅")

print(f"\n🚀 Next Steps:")
print(f"  1. Push code to GitHub repository")
print(f"  2. Set up GitHub secrets (HF_TOKEN)")
print(f"  3. GitHub Actions will automatically:")
print(f"     - Train the model")
print(f"     - Deploy to Hugging Face Spaces")
print(f"  4. Access deployed app at: https://huggingface.co/spaces/Mritula123/Mlmodeldrug")

print(f"\n✅ Complete ML pipeline validation successful! 🎉")